In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from skimage.transform import warp, AffineTransform

gesture_folders = ['six', 'seven', 'eight', 'nine', 'ten']

for folder in gesture_folders:
    folder_path = f'/content/drive/My Drive/Colab Notebooks/hand_gesture/{folder}'
    output_folder_path = f'/content/drive/My Drive/Colab Notebooks/hand_gesture/{folder}'

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    def augment_image(image):
        augmented_images = []

        # 좌우로 15도, 30도 회전
        for angle in [-15, 15, -30, 30]:
            rows, cols = image.shape[:2]
            M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
            rotated_image = cv2.warpAffine(image, M, (cols, rows))
            augmented_images.append(rotated_image)

        # 뒤집은 이미지
        flipped_image = cv2.flip(image, 1)
        augmented_images.append(flipped_image)

        # 뒤집은 이미지에 회전 추가
        for angle in [-15, 15, -30, 30]:
            rows, cols = flipped_image.shape[:2]
            M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
            rotated_flipped_image = cv2.warpAffine(flipped_image, M, (cols, rows))
            augmented_images.append(rotated_flipped_image)

        # 확대
        zoom_factor = 1.1
        zoomed_image = cv2.resize(image, (0, 0), fx=zoom_factor, fy=zoom_factor)
        augmented_images.append(zoomed_image)

        # 뒤집은 이미지 확대
        flipped_zoomed_image = cv2.resize(flipped_image, (0, 0), fx=zoom_factor, fy=zoom_factor)
        augmented_images.append(flipped_zoomed_image)

        return augmented_images

    for i in range(1, 11):
        filename = f'{folder}_{i}.jpg'
        image_path = os.path.join(folder_path, filename)

        original_image = cv2.imread(image_path)

        augmented_images = augment_image(original_image)

        for j, aug_image in enumerate(augmented_images):
            output_filename = f'{folder}_{i}_{j+1}.jpg'
            output_path = os.path.join(output_folder_path, output_filename)

            cv2.imwrite(output_path, aug_image, [int(cv2.IMWRITE_JPEG_QUALITY), 95])


In [2]:
import os
import numpy as np
from PIL import Image

gesture_folders = ['six', 'seven', 'eight', 'nine', 'ten']

image_size = (224, 224)

X = []
y = []

for folder in gesture_folders:
    image_dir = f'/content/drive/My Drive/Colab Notebooks/hand_gesture/{folder}'

    label = gesture_folders.index(folder)

    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(image_dir, filename)

            image = Image.open(image_path).resize(image_size)

            image_array = np.array(image)

            X.append(image_array)
            y.append(label)

X = np.array(X)
y = np.array(y)

print(f'이미지: {X.shape}')
print(f'라벨: {y.shape}')


이미지: (600, 224, 224, 3)
라벨: (600,)


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X = np.array(X)
y = np.array(y)

unique_labels = np.unique(y)

X_train = []
y_train = []
X_val = []
y_val = []
X_test = []
y_test = []

for label in unique_labels:
    label_idx = np.where(y == label)[0]
    X_label = X[label_idx]
    y_label = y[label_idx]

    X_label_train, X_label_test, y_label_train, y_label_test = train_test_split(X_label, y_label, test_size=0.2, random_state=77)

    X_label_train, X_label_val, y_label_train, y_label_val = train_test_split(X_label_train, y_label_train, test_size=0.2, random_state=77)

    X_train.extend(X_label_train)
    y_train.extend(y_label_train)
    X_val.extend(X_label_val)
    y_val.extend(y_label_val)
    X_test.extend(X_label_test)
    y_test.extend(y_label_test)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

y_train = to_categorical(y_train, num_classes=5)
y_val = to_categorical(y_val, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

print(f'학습 데이터 : {X_train.shape}, {y_train.shape}')
print(f'검증 데이터 : {X_val.shape}, {y_val.shape}')
print(f'테스트 데이터 : {X_test.shape}, {y_test.shape}')


학습 데이터 : (380, 224, 224, 3), (380, 5)
검증 데이터 : (100, 224, 224, 3), (100, 5)
테스트 데이터 : (120, 224, 224, 3), (120, 5)


In [4]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential, layers

# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

# VGG16 모델 구조 사용
base_model = VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1)

model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=100,
          validation_data=(X_val, y_val),
          callbacks=[early_stopping, reduce_lr])


# 테스트
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


Epoch 1/100
12/12 [==============================] - 54s 2s/step - loss: 2.2702 - accuracy: 0.2158 - val_loss: 422.3661 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 2/100
12/12 [==============================] - 6s 523ms/step - loss: 2.2488 - accuracy: 0.1974 - val_loss: 99.5094 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 3/100
12/12 [==============================] - 6s 488ms/step - loss: 2.1867 - accuracy: 0.1816 - val_loss: 114.7754 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 4/100
12/12 [==============================] - 7s 530ms/step - loss: 2.0125 - accuracy: 0.2342 - val_loss: 39.1867 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 5/100
12/12 [==============================] - 7s 611ms/step - loss: 2.0436 - accuracy: 0.2447 - val_loss: 22.6073 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 6/100
12/12 [==============================] - 6s 490ms/step - loss: 2.1146 - accuracy: 0.1711 - val_loss: 50.9369 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 7/100
12/12 [==============================] - 6s 51

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential, layers

# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

# VGG16 모델 구조 사용
base_model = VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1)

model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=100,
          validation_data=(X_val, y_val),
          callbacks=[early_stopping, reduce_lr])

# 테스트
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


Epoch 1/100
12/12 [==============================] - 53s 2s/step - loss: 2.1865 - accuracy: 0.2421 - val_loss: 176.8994 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 2/100
12/12 [==============================] - 6s 501ms/step - loss: 2.0532 - accuracy: 0.2211 - val_loss: 146.5332 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 3/100
12/12 [==============================] - 7s 614ms/step - loss: 2.0790 - accuracy: 0.2289 - val_loss: 25.2175 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 4/100
12/12 [==============================] - 7s 523ms/step - loss: 1.9708 - accuracy: 0.2211 - val_loss: 71.8568 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 5/100
12/12 [==============================] - 6s 494ms/step - loss: 1.9225 - accuracy: 0.2395 - val_loss: 49.6059 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 6/100
12/12 [==============================] - 8s 629ms/step - loss: 1.9507 - accuracy: 0.2105 - val_loss: 2.5953 - val_accuracy: 0.2300 - lr: 0.0010
Epoch 7/100
12/12 [==============================] - 7s 524

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential, layers

# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# ResNet50 모델 구조 가져오기
resnet50 = ResNet50(weights=None, include_top=False, input_shape=(224, 224, 3))
resnet50.trainable = False

model = Sequential()
model.add(resnet50)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

# 모델 컴파일
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 콜백 함수 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1)
checkpoint = ModelCheckpoint('resnet50_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True)

# 모델 학습
model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# 테스트
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


Epoch 1/100
12/12 [==============================] - ETA: 0s - loss: 111.9194 - accuracy: 0.1921
Epoch 1: val_accuracy improved from -inf to 0.20000, saving model to resnet50_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12/12 [==============================] - 58s 2s/step - loss: 111.9194 - accuracy: 0.1921 - val_loss: 101716664.0000 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 2/100
12/12 [==============================] - ETA: 0s - loss: 16.8536 - accuracy: 0.2132
Epoch 2: val_accuracy did not improve from 0.20000
12/12 [==============================] - 6s 493ms/step - loss: 16.8536 - accuracy: 0.2132 - val_loss: 327955040.0000 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 3/100
12/12 [==============================] - ETA: 0s - loss: 10.7600 - accuracy: 0.2553
Epoch 3: val_accuracy did not improve from 0.20000
12/12 [==============================] - 6s 481ms/step - loss: 10.7600 - accuracy: 0.2553 - val_loss: 3417166.7500 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 4/100
12/12 [==============================] - ETA: 0s - loss: 11.5402 - accuracy: 0.2211
Epoch 4: val_accuracy did not improve from 0.20000
12/12 [==============================] - 6s 482ms/step - loss: 11.5402 - accuracy: 0.2211 - val_loss: 2

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential, layers

# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# ResNet50 모델 구조 가져오기
resnet50 = ResNet50(weights=None, include_top=False, input_shape=(224, 224, 3))
resnet50.trainable = False

model = Sequential()
model.add(resnet50)
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

# 모델 컴파일
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 콜백 함수 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1)
checkpoint = ModelCheckpoint('resnet50_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True)

# 모델 학습
model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr, checkpoint]
)


# 테스트
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


Epoch 1/100
12/12 [==============================] - ETA: 0s - loss: 2.1401 - accuracy: 0.2026
Epoch 1: val_accuracy improved from -inf to 0.20000, saving model to resnet50_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12/12 [==============================] - 12s 487ms/step - loss: 2.1401 - accuracy: 0.2026 - val_loss: 48.0131 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 2/100
12/12 [==============================] - ETA: 0s - loss: 2.0746 - accuracy: 0.2474
Epoch 2: val_accuracy did not improve from 0.20000
12/12 [==============================] - 6s 461ms/step - loss: 2.0746 - accuracy: 0.2474 - val_loss: 17.0599 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 3/100
12/12 [==============================] - ETA: 0s - loss: 2.0312 - accuracy: 0.2605
Epoch 3: val_accuracy did not improve from 0.20000
12/12 [==============================] - 5s 376ms/step - loss: 2.0312 - accuracy: 0.2605 - val_loss: 27.2052 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 4/100
12/12 [==============================] - ETA: 0s - loss: 1.9790 - accuracy: 0.2632
Epoch 4: val_accuracy did not improve from 0.20000
12/12 [==============================] - 6s 522ms/step - loss: 1.9790 - accuracy: 0.2632 - val_loss: 16.4358 - val_accuracy: 0